In [1]:
import ROOT as r
import math
import array
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from math import pi

#______________Add include directory_______________
current_dir = os.getcwd()
include_dir = os.path.join(current_dir, '../include')
sys.path.insert(0, include_dir)
#__________________________________________________
import ERROR
import CONFIG
import DBPARSE
from ASYMMETRY import *
from UTILITIES import *
from SIMFITS import DistributionFits
from ROOT import gStyle, TChain, TH1F, TCanvas, TLegend
import CORRECTIONS
import CALCULATE
from joblib import Parallel, delayed
import importlib
#importlib.reload(ASYMMETRY)

Welcome to JupyROOT 6.26/06


In [2]:
pas="1"
config="4"

In [3]:
muN=1.9103 #minus

In [4]:
def counts_within_ellipse(x_edges, y_edges, bin_contents, dxmin, dxmax, dymin, dymax):
    # Calculate ellipse parameters
    
    x_center = (dxmin + dxmax) / 2
    y_center = (dymin + dymax) / 2
    a = (dxmax - dxmin) / 2
    b = (dymax - dymin) / 2
    
    total_counts = 0

    for i in range(len(x_edges) - 1):
        
        for j in range(len(y_edges) - 1):
            # Calculate bin center
            x_center_bin = (x_edges[i] + x_edges[i + 1]) / 2
            y_center_bin = (y_edges[j] + y_edges[j + 1]) / 2
            
            # Check if bin center is within ellipse
            if ((x_center_bin - x_center) / a) ** 2 + ((y_center_bin - y_center) / b) ** 2 <= 1:
                total_counts += bin_contents[i][j]
    #X, Y = np.meshgrid(x_edges, y_edges)
    #plt.pcolormesh(X, Y, bin_contents.T, shading='auto')
   # plt.colorbar(label='Counts')

   ## Drawing the ellipse
   # x_center = (dxmin + dxmax) / 2
   # y_center = (dymin + dymax) / 2
   # a = (dxmax - dxmin) / 2
   # b = (dymax - dymin) / 2

   # ellipse = Ellipse((x_center, y_center), width=2*a, height=2*b, edgecolor='r', facecolor='none', linewidth=2)
   # plt.gca().add_patch(ellipse)
   # plt.show()
    return total_counts

In [5]:
import sys
import re

class Filter:
    def __init__(self):
        self.old_stderr = sys.stderr
        sys.stderr = self

    def write(self, message):
        # Change the pattern here to match the specific ROOT errors
        if not re.search(r"Error in <TList::Clear>:", message):
            self.old_stderr.write(message)

    def flush(self):
        pass

# Apply the filter
filter = Filter()



# Set Config

In [6]:
#in GeV^2
m=.939565
muN=-1.9103
if config=="2":
    Q2=3
    tau=Q2/(4*m**2)
    theta=29.5*pi/180
if config=="3":
    Q2=6.83
    #Q2=6.62 #seans number
    tau=Q2/(4*m**2)
    theta=36.5*pi/180
if config=="4":
    Q2=9.82
    tau=Q2/(4*m**2)
    theta=35*pi/180

In [7]:
W2min=CONFIG.Function_JSON("W2min",f"../config/cuts{config}.cfg")
W2max=CONFIG.Function_JSON("W2max",f"../config/cuts{config}.cfg")
dxmin=CONFIG.Function_JSON("dxmin",f"../config/cuts{config}.cfg")
dxmax=CONFIG.Function_JSON("dxmax",f"../config/cuts{config}.cfg")
dymin=CONFIG.Function_JSON("dymin",f"../config/cuts{config}.cfg")
dymax=CONFIG.Function_JSON("dymax",f"../config/cuts{config}.cfg")
dybgmin=CONFIG.Function_JSON("dybgmin",f"../config/cuts{config}.cfg")
dybgmax=CONFIG.Function_JSON("dybgmax",f"../config/cuts{config}.cfg")
coinmin=CONFIG.Function_JSON("coinmin",f"../config/cuts{config}.cfg")
coinmax=CONFIG.Function_JSON("coinmax",f"../config/cuts{config}.cfg")
Q2=CONFIG.Function_JSON("Q2",f"../config/cuts{config}.cfg")

#  "dxmin"	: -0.481,
#  "dxmax"	: 0.498, old gen2


print(f'W2min: {W2min}')
print(f'W2max: {W2max}')
print(f'dxmin: {dxmin}')
print(f'dxmax: {dxmax}')
print(f'dymin: {dymin}')
print(f'dymax: {dymax}')
print(f'dybgmin: {dybgmin}')
print(f'dybgmax: {dybgmax}')
print(f'Q2: {Q2}')
print(f'Coin Min: {coinmin} Coin Max: {coinmax}')

W2min: -1
W2max: 1.9
dxmin: -0.403
dxmax: 0.406
dymin: -0.98
dymax: 0.98
dybgmin: -1.3
dybgmax: 1.2
Q2: 9.82
Coin Min: 81 Coin Max: 100


In [8]:
cut="w2"
value=[W2min,W2max]


In [9]:
def Function_APHYSSENS(config,cut,value,cutstyle=0):
    #cutstyle: 0 for rectangle, 1 for ellipse
    import ROOT as r
    import math
    import array
    import os
    import sys
    import matplotlib.pyplot as plt
    import numpy as np

    #______________Add include directory_______________
    current_dir = os.getcwd()
    include_dir = os.path.join(current_dir, '../include')
    sys.path.insert(0, include_dir)
    #__________________________________________________
    import ERROR
    import CONFIG
    import DBPARSE
    import ASYMMETRY 
    import UTILITIES 
    from SIMFITS import DistributionFits
    from ROOT import gStyle, TChain, TH1F, TCanvas, TLegend
    import CALCULATE
    import CORRECTIONS
    from joblib import Parallel, delayed
    if cutstyle==0:    #cuts
        W2min=CONFIG.Function_JSON("W2min",f"../config/cuts{config}.cfg")
        W2max=CONFIG.Function_JSON("W2max",f"../config/cuts{config}.cfg")
        dxmin=CONFIG.Function_JSON("dxmin",f"../config/cuts{config}.cfg")
        dxmax=CONFIG.Function_JSON("dxmax",f"../config/cuts{config}.cfg")
        dymin=CONFIG.Function_JSON("dymin",f"../config/cuts{config}.cfg")
        dymax=CONFIG.Function_JSON("dymax",f"../config/cuts{config}.cfg")
        dybgmin=CONFIG.Function_JSON("dybgmin",f"../config/cuts{config}.cfg")
        dybgmax=CONFIG.Function_JSON("dybgmax",f"../config/cuts{config}.cfg")
        coinmin=CONFIG.Function_JSON("coinmin",f"../config/cuts{config}.cfg")
        coinmax=CONFIG.Function_JSON("coinmax",f"../config/cuts{config}.cfg")
    if cutstyle==1:
        W2min=CONFIG.Function_JSON("W2min",f"../config/cuts{config}ellipse.cfg")
        W2max=CONFIG.Function_JSON("W2max",f"../config/cuts{config}ellipse.cfg")
        dxmin=CONFIG.Function_JSON("dxmin",f"../config/cuts{config}ellipse.cfg")
        dxmax=CONFIG.Function_JSON("dxmax",f"../config/cuts{config}ellipse.cfg")
        dymin=CONFIG.Function_JSON("dymin",f"../config/cuts{config}ellipse.cfg")
        dymax=CONFIG.Function_JSON("dymax",f"../config/cuts{config}ellipse.cfg")
        dybgmin=CONFIG.Function_JSON("dybgmin",f"../config/cuts{config}ellipse.cfg")
        dybgmax=CONFIG.Function_JSON("dybgmax",f"../config/cuts{config}ellipse.cfg")
        coinmin=CONFIG.Function_JSON("coinmin",f"../config/cuts{config}ellipse.cfg")
        coinmax=CONFIG.Function_JSON("coinmax",f"../config/cuts{config}ellipse.cfg")
    #____________Varying Cut__________________
    if cut=="w2":
        W2max=value[1]
        W2min=value[0]
    elif cut=="dy":
        dymax=value[1]
        dymin=value[0]
    elif cut=="dx":
        dxmax=value[1]
        dxmin=value[0]        
    elif cut=="coin":
        coinmin=value[1]
        coinmax=value[0]
    quasicut=[[coinmin,coinmax],[W2min,W2max],[dxmin,dxmax],[dymin,dymax]]
    
    
    dxmin=CONFIG.Function_JSON("dxmin",f"../config/cuts{config}.cfg")
    dxmax=CONFIG.Function_JSON("dxmax",f"../config/cuts{config}.cfg")
    lower_bound=dxmin
    upper_bound=dxmax
    Raw=ASYMMETRY.Function_ASYMMETRYSENS(config,cut,value,cutstyle)
    print("Raw Asymmetry Calculated")
    if cutstyle==0:
        print("Corrections with rectangular cut.")
        Fit=ASYMMETRY.Function_FITDXSENS(config,cut,value)
    
    
    
        data,bg,totalFit,simp,simn=Fit
        bin_centers,bin_contents=bg
        numBG=np.sum(bin_contents[(bin_centers >= lower_bound) & (bin_centers <= upper_bound)])
        bin_centers,bin_contents=totalFit
        numTotal=np.sum(bin_contents[(bin_centers >= lower_bound) & (bin_centers <= upper_bound)])
        bin_centers,bin_contents=simp
        numP=np.sum(bin_contents[(bin_centers >= lower_bound) & (bin_centers <= upper_bound)])
        bin_centers,bin_contents=simn
        Y=np.sum(bin_contents[(bin_centers >= lower_bound) & (bin_centers <= upper_bound)])
        print("dx Histogram Fit Completed")
        accResult,bgResult,fp=CORRECTIONS.Function_ANALYZEROOTFILE(config,bg,simp,totalFit)
    if cutstyle==1:
        print("Corrections with elliptical cut.")
        Fit=ASYMMETRY.Function_FITDXSENS2D(config,cut,value)
        x_edges1, y_edges1, bin_contents1 = Fit[0] #data
        x_edges2, y_edges2, bin_contents2 = Fit[2] #fit
        x_edges3, y_edges3, bin_contents3 = Fit[4] #neutron
        x_edges4, y_edges4, bin_contents4 = Fit[1] #background
        x_edges5, y_edges5, bin_contents5 = Fit[3] #proton
    
        numTotal = counts_within_ellipse(x_edges1, y_edges1, bin_contents1, dxmin, dxmax, dymin, dymax)
        Y = counts_within_ellipse(x_edges3, y_edges3, bin_contents3, dxmin, dxmax, dymin, dymax)
        numBG = counts_within_ellipse(x_edges4, y_edges4, bin_contents4, dxmin, dxmax, dymin, dymax)
        numP = counts_within_ellipse(x_edges5, y_edges5, bin_contents5, dxmin, dxmax, dymin, dymax)
    
        accResult,bgResult,fp=CORRECTIONS.Function_ANALYZEROOTFILE(config,numBG,numP,numTotal,cutstyle=1)
    
    
    
    
    
    #bin_centers,bin_contents=simn
    #Y=np.sum(bin_contents[(bin_centers >= lower_bound) & (bin_centers <= upper_bound)])
    TrueYield=Raw[3]
    print("f A combinations Determined")
    
    Phys=ASYMMETRY.Function_APHYS(config,pas,Raw,accResult,bgResult,fp)
    print("Aphys Calculated")
    
    weighted_A,weighted_A_E,fbackg,rawA,rawAE,totalSys,totalStat=Phys
    print('Returned: Aphys, Aphys_E, fbg, cut, cutvalue')
    R,RE=CALCULATE.Function_CALCGEN(config,weighted_A,weighted_A_E)
    return weighted_A,weighted_A_E,fbackg,cut,value,Y,R,RE,rawA,rawAE,np.mean(Raw[1]),quasicut,totalSys,totalStat



In [10]:
Results=Function_APHYSSENS(config,cut,value,0)

Config: 4
Raw Asymmetry Calculated
Corrections with rectangular cut.
GEN4
dx Histogram Fit Completed
f A combinations Determined
fA Error:0.0027120517730110844+-0.0014158513713036418
f Error:0.8077+-0.009224517961227639
Aphys Calculated
Returned: Aphys, Aphys_E, fbg, cut, cutvalue


In [11]:
A=Results[0]
AE=Results[1]
rawA=Results[8]
rawAE=Results[9]
R=abs(Results[6])
RE=Results[7]
quasicut=Results[11]
systematic=Results[12]
statistical=Results[13]
print(f"Kin{config} - muN GEN/GMN: {abs(R*muN)}+/-{abs(RE*muN)}")

#np.save(f'SavedArrays/GEN{config}/Pass{pas}/GEGM.npy',R)
#np.save(f'SavedArrays/GEN{config}/Pass{pas}/GEGME.npy',RE)

Kin4 - muN GEN/GMN: 0.8928728043983359+/-0.5378854622415686


In [29]:
print(f"GEN{config}")
print(f"Araw: {rawA:.4f}+-{rawAE:.4f}")
print(f"Aphys: {A:.4f}+-{AE:.4f}")


GEN4
Araw: -0.0113+-0.0081
Aphys: -0.1878+-0.1079


In [108]:
print(f'Cuts: Coin{quasicut[0]} W2{quasicut[1]} dX{quasicut[2]} dY{quasicut[3]}')

Cuts: Coin[95, 106] W2[0, 1.5] dX[-0.481, 0.498] dY[-0.822, 0.8]


Kin2 - muN GEN/GMN: 0.4847315879194152+/-0.035786597796588764

Elliptical - Kin3 - muN GEN/GMN: 0.8171+/-0.1734

Standard - Kin3 - muN GEN/GMN: 0.8075+/-0.1759

Kin4 - muN GEN/GMN: 0.8928728043983359+/-0.5378854622415686

In [117]:
.17338/.817075

0.2121959428449041

In [118]:
.17586/.80746

0.21779407029450376

In [110]:
GEGM4E=np.sum(np.load(f'SavedArrays/GEN4/Pass{pas}/GEGME.npy'))

In [111]:
GEGM4E=np.sum(np.load(f'SavedArrays/GEN4/Pass{pas}/GEGME.npy'))

In [112]:
GEGM4E

0.2815711994145258

\begin{equation}
    a = A_{\perp}
\end{equation}

\begin{equation}
b = 2\sqrt{\tau(\tau+1)}\tan{\frac{\theta}{2}}
\end{equation}

\begin{equation}
c = A_{\perp} \left(\tau + 2\tau(1+\tau)\tan^2\frac{\theta}{2}\right)
\end{equation}


In [113]:
a=A
b=2*(np.sqrt(tau*(tau+1))*np.tan(theta/2))
c=A*(tau+2*tau*(1+tau)*np.tan(theta/2)**2)
aE=AE
cE=AE*(tau+2*tau*(1+tau)*np.tan(theta/2)**2)


In [114]:
def RAT(a, b, c, aE, cE):
    R = (-b + np.sqrt(b**2 - 4 * a * c)) / (2 * a)
    
    # Calculate the partial derivatives
    p1=(-b**2)/np.sqrt(b**2-4*a*c)
    p2=(2*a*c)/np.sqrt(b**2-4*a*c)
    p3=b
    partial_a = (p1+p2+p3)/(2*a**2)
    partial_c = -1 / (np.sqrt(b**2 - 4 * a * c))
    
    # Calculate the propagated error
    RE = np.sqrt((partial_a * aE)**2 + (partial_c * cE)**2)
    
    return R,RE


In [115]:
print(f'Cuts: Coin{quasicut[0]} W2{quasicut[1]} dX{quasicut[2]} dY{quasicut[3]}')

Cuts: Coin[95, 106] W2[0, 1.5] dX[-0.481, 0.498] dY[-0.822, 0.8]
